# A/B тестирование рекомендательной системы

Целью данного исследования является провести анализ результатов A/B-теста:

* Оценить корректность проведения теста


* Проанализировать результаты теста


**Описание теста:**


* Название теста: `recommender_system_test`;


* Группы: А — контрольная, B — новая платёжная воронка;


* Дата запуска: 2020-12-07;


* Дата остановки набора новых пользователей: 2020-12-21;


* Дата остановки: 2021-01-04;


* Аудитория: 15% новых пользователей из региона EU;


* Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;


* Ожидаемое количество участников теста: 6000;


* Ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:

    * конверсии в просмотр карточек товаров — событие `product_page`;
    
    * просмотры корзины — `product_cart`;
    
    * покупки — `purchase`;


**Структура данных:**


Календарь маркетинговых событий хранится в файле `ab_project_marketing_events.csv`:

* `name` – название маркетингового события;


* `regions` – регионы, в которых будет проводиться рекламная кампания;


* `start_dt` – дата начала кампании;


* `finish_dt` – дата завершения кампании;

Данные с пользователями, зарегистрировавшимися с 7 по 21 декабря 2020 года, находятся в файле `final_ab_new_users.csv`:

* `user_id` – уникальный идентификатор пользователя;


* `first_date` – дата регистрации;


* `region` – регион пользователя;


* `device` – устройство, с которого проходила регистрация;

Данные о действиях, совершенными новыми пользователями с 7 декабря 2020 по 4 января 2021 года, хранятся в файле `final_ab_events`:

* `user_id` – уникальный идентификатор пользователя;


* `event_dt` – дата и время покупки;


* `event_name` – тип действия;


* `details` – дополнительные сведения о действиях. Например, для события покупки `purchase` в этом поле хранится стоимость покупки в долларах.

Информация об участниках теста записана в файле `final_ab_participants`:

* `user_id` – уникальный идентификатор пользователя;


* `group` – группа пользователя;


* `ab_test` – название теста;


**План:**

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Setup" data-toc-modified-id="Setup-2">Setup</a></span></li>
        <li>
            <span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-3">Предобработка данных</a></span>
            <ul class="toc-item">
                <li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-3.1">Обработка пропусков</a></span></li>
                <li><span><a href="#Преобразование-типов-данных" data-toc-modified-id="Преобразование-типов-данных-3.2">Преобразование типов данных</a></span></li>
                <li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-3.3">Обработка дубликатов</a></span></li>
                <li><span><a href="#Предобработка-пользователей" data-toc-modified-id="Предобработка-пользователей-3.4">Предобработка пользователей</a></span></li>
                <li><span><a href="#Итоги-предобработки" data-toc-modified-id="Итоги-предобработки-3.5">Итоги предобработки</a></span></li>
            </ul>
        </li>
        <li>
            <span><a href="#Исследовательский-анализ-данных" data-toc-modified-id="Исследовательский-анализ-данных-4">Исследовательский анализ данных</a></span>
            <ul class="toc-item">
                <li><span><a href="#Воронка-событий" data-toc-modified-id="Воронка-событий-4.1">Воронка событий</a></span></li>
                <li><span><a href="#Количество-событий-на-одного-пользователя" data-toc-modified-id="Количество-событий-на-одного-пользователя-4.2">Количество событий на одного пользователя</a></span></li>
                <li><span><a href="#Выручка-на-покупателя" data-toc-modified-id="Выручка-на-покупателя-4.3">Выручка на покупателя</a></span></li>
                <li><span><a href="#Итоги-по-анализу-данных" data-toc-modified-id="Итоги-по-анализу-данных-4.4">Итоги по анализу данных</a></span></li>
            </ul>
        </li>
        <li>
            <span><a href="#Проверка-гипотез" data-toc-modified-id="Проверка-гипотез-5">Проверка гипотез</a></span>
            <ul class="toc-item">
                <li><span><a href="#Конверсия-пользователей-в-покупатели" data-toc-modified-id="Конверсия-пользователей-в-покупатели-5.1">Конверсия пользователей в покупатели</a></span></li>
                <li><span><a href="#Выручка-на-пользователя" data-toc-modified-id="Выручка-на-пользователя-5.2">Выручка на пользователя</a></span></li>
            </ul>
        </li>
        <li><span><a href="#Итог" data-toc-modified-id="Итог-6">Итог</a></span></li>
    </ul>
</div>

# Setup

In [ ]:
%%bash
if ! test -f "custom_plotly_templates.py"; then
    wget https://gist.githubusercontent.com/rusmux/a74e5060a470f45e0f2d52b2484a1ecf/raw/8e1b62dacd5aadcd41cca9658c5a7716d4bdc0d5/custom_plotly_templates.py &> /dev/null
    echo "Successfully downloaded custom plotly templates."
fi

In [ ]:
import custom_plotly_templates
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from custom_plotly_templates import custom_show_config
from ipywidgets import fixed, interact
from plotly.subplots import make_subplots
from scipy import stats

In [ ]:
pd.set_option("display.float_format", "{:.2f}".format)
pio.templates.default = "plotly_white+custom_white"
pio.renderers["notebook"].config.update(custom_show_config)

# Предобработка данных

In [ ]:
marketing_events = pd.read_csv(
    "https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv", parse_dates=["start_dt", "finish_dt"]
)
new_users = pd.read_csv("https://code.s3.yandex.net/datasets/final_ab_new_users.csv", parse_dates=["first_date"])
events = pd.read_csv(
    "https://code.s3.yandex.net/datasets/final_ab_events.csv", parse_dates=["event_dt"], infer_datetime_format=True
)
participants = pd.read_csv("https://code.s3.yandex.net/datasets/final_ab_participants.csv")

df_names = ["marketing_events", "new_users", "events", "participants"]

In [ ]:
interact(lambda df: globals()[df], df=df_names);

Среди маркетинговых событий на наше тестирование потенциально мог повлиять Новый год.

In [ ]:
interact(lambda df: globals()[df].info(memory_usage="deep"), df=df_names);

В таблице `events` в столбце `details` отсутствует больше 85% значений. Во всех остальных таблицах явных пропусков нет.

In [ ]:
interact(lambda df: globals()[df].describe(datetime_is_numeric=True), df=df_names);

Новые пользователи приходили с 7 по 23 декабря, а вот участников набирали действительно только по 21 декабря. Странно, что последний пользователь пришел к нам только 23 декабря. Можно предположить, что наш сайт не очень популярен, но за 16 дней у нас зарегистрировалось почти 62 тысячи новых пользователей по всему миру – больше, чем 3,5 тысячи новых пользователей в день. Больше похоже на то, что это либо ошибка в сборе данных, либо у нас неполные данные.

Последнее событие датируется 30 декабря 2020 года, что тоже весьма странно, так как тест завершился только 4 января 2021 года. Возможно, после Нового года, в первых числах января, людям было не до этого, но это тоже подозрительно – за 16 дней у нас ~440 тысяч событий, то есть по ~27 тысяч событий в день. В любом случае с момента регистрации последнего пользователя прошло меньше 14 дней.

Неполные данные могут поставить под сомнения корректность теста, так как с полными данными статистическая значимость теста может поменяться.

В таблице с участниками `participants` видно, что у нас участники с двух тестов.

## Обработка пропусков

Снова посмотрим на явные пропущенные значения.

In [ ]:
interact(lambda df: globals()[df].isna().mean(), df=df_names);

Только в таблице `events` в столбце `details` есть пропущенные значения. Они означают, что дополнительной информации о событии нет, так что отбрасывать или заполнять их не будем. Посмотрим, для каких событий указан параметр `details`.

In [ ]:
events.groupby("event_name")["details"].apply(lambda group: group.isna().mean())

Поле `details` отсутствует для всех событий, кроме покупок, значит, в столбце `details` указана только стоимость покупок.

Посмотрим на неявные пропуски.

In [ ]:
new_users["region"].unique()

Всего 4 региона – Евросоюз, Северная Америка, Азиатско-Тихоокеанский регион и СНГ. Неявных пропусков нет.

In [ ]:
new_users["device"].unique()

Устройств тоже 4 и неявных пропусков нет.

In [ ]:
events["event_name"].unique()

Событий тоже 4, неявных пропусков нет.

In [ ]:
participants["group"].unique()

Тестовых групп 2.

In [ ]:
participants["ab_test"].unique()

В строковых столбцах нет неявных пропусков. Посмотрим, есть ли неявные пропуски в идентификаторе пользователей.

In [ ]:
for df in [new_users, events, participants]:
    print(df["user_id"].str.contains("^[^[1-9]]*$").sum() + df["user_id"].str.contains("^[^[A-z]]*$").sum())

Во всех 3 таблицах нет идентификаторов, в которых только цифры или только буквы, значит, неявных пропусков нет.

## Преобразование типов данных

В столбцах с регионом и устройством пользователя всего по 4 различных значения. Имеет смысл преобразовать столбцы с малым количеством уникальных значений к категориальному типу данных.

In [ ]:
new_users["region"] = new_users["region"].astype("category")
new_users["device"] = new_users["device"].astype("category")
events["event_name"] = events["event_name"].astype("category")
participants["group"] = participants["group"].astype("category")
participants["ab_test"] = participants["ab_test"].astype("category")

Посмотрим, сколько уникальных пользователей в таблице `events`.

In [ ]:
events["user_id"].nunique() / len(events)

Уникальных пользователей всего 13% от общего количества записей, так что приведем идентификатор к категориальному типу.

In [ ]:
events["user_id"] = events["user_id"].astype("category")

Посмотрим, сколько уникальных пользователей в таблице `participants`, так как один пользователь может присутствовать в нескольких тестах.

In [ ]:
participants["user_id"].nunique() / len(participants)

Уникальных пользователей больше 90% от общего числа записей, так что приводить к категориальному типу не имеет смысла.

## Обработка дубликатов

In [ ]:
new_users.duplicated(["user_id"]).sum()

In [ ]:
participants.duplicated(["user_id", "ab_test"]).sum()

Среди новых пользователей дубликатов нет. В каждом A/B-тесте все пользователи уникальны. Посмотрим на события.

In [ ]:
events.duplicated(["user_id", "event_dt", "event_name"]).sum()

Одинаковых событий в одно и то же время у пользователей нет. Посмотрим, есть ли вообще события, происходящие в одно время.

In [ ]:
events.duplicated(["user_id", "event_dt"]).sum() / len(events)

Почему-то 34% событий произошли в одно время. Посмотрим на них поближе.

In [ ]:
events[events.duplicated(["user_id", "event_dt"], keep=False)].sort_values(
    by=["user_id", "event_dt", "event_name"]
).head(3)

Пользователь за одну секунду зашел в свой аккаунт, перешел на страницу продукта, а потом купил. И все это за одну секунду. Весьма странно. Вероятно, это ошибка в сборе данных, и всем события одной сессии присваивается время последнего действия. Это не очень хорошо, потому что нельзя установить точную последовательность действий пользователя.

Неявных дубликатов нет, как было видно выше в разделе [обработки пропусков](#Обработка-пропусков).

## Предобработка пользователей

Посмотрим, сколько уникальных участников тестов, и сколько новых пользователей вообще.

In [ ]:
print("Уникальных участников тестов:", participants["user_id"].nunique())
print("Уникальных пользователей:", new_users["user_id"].nunique())

Примерно каждый 4 стал участником какого-либо теста. Проверим, есть ли пользователи, которые присутствуют в обоих тестах.

In [ ]:
users_duplicated = participants[participants.duplicated(["user_id"])]["user_id"]
len(users_duplicated)

Такие пользователи есть. Их надо исключить из анализа, так как нельзя сказать, какое именно изменение повлияло на их поведение. Но исключать можно не всех пользователей, а лишь тех, кто попал в тестовую группу другого эксперимента, потому что участники контрольной группы не подвергались никаким изменениям. 

In [ ]:
participants.query("ab_test == 'interface_eu_test' and user_id in @users_duplicated")["group"].value_counts()

Больше половины продублированных пользователей попали в контрольную группу, и их можно не удалять.

In [ ]:
users_duplicated = participants.query(
    "user_id in @users_duplicated and ab_test == 'interface_eu_test' and group == 'B'"
)["user_id"]

In [ ]:
participants = participants.query("ab_test == 'recommender_system_test'")
total_participants = len(participants)
participants = participants[~participants["user_id"].isin(users_duplicated)].drop("ab_test", axis=1)

print(f"Относительная потеря пользователей: {len(users_duplicated) / total_participants:2.2%}")

Добавим в таблицу с участниками теста данные о пользователях.

In [ ]:
participants = participants.merge(new_users, on="user_id")

В [проверке на дубликаты](#Обработка-дубликатов) мы выяснили, что в каждом тесте каждый пользователь уникален, так что каждому пользователю присвоена только одна группа теста. Проверим регион участников тестирования – они должны составлять 15% новых пользователей с региона EU.

In [ ]:
participants["region"].value_counts(normalize=True)

Среди участников присутствуют пользователи и с других регинов. Нам нужны только пользователи из стран Евросоюза, так что отбросим других пользователей.

In [ ]:
users_other_regions = participants.query("region != 'EU'")["user_id"]
participants = participants[~participants["user_id"].isin(users_other_regions)].drop("region", axis=1)

print(f"Относительная потеря пользователей: {len(users_other_regions) / total_participants:2.2%}")

Посмотрим на количество участников теста после предобработки данных.

In [ ]:
eu_users_fraction = len(participants) / len(new_users.query("first_date <= '2020-12-21' and region == 'EU'"))
print("Абсолютное количество участников теста:", len(participants))
print(f"Доля от новых пользователей стран Евросоюза: {eu_users_fraction:.2%}")

Количество участников на ~400 меньше предполагаемых 6 тысяч, и они составляют 13% новых пользователей стран Евросоюза. Это немного меньше, чем предполагаемые 15%, но все же может негативно повлиять на статистическую значимость результатов теста.

Посмотрим на распределение пользователей по группам теста после предобработки.

In [ ]:
participants["group"].value_counts()

В контрольной группе на 34% больше участников, чем в тестовой. Тоже может негативно сказаться на статистической значимости результатов теста.

Для удобства объединим данные о пользователях и событиях в одну таблицу.

In [ ]:
events = events.merge(participants, on="user_id")
events.head()

Посмотрим, все ли пользователи совершали события.

In [ ]:
events["user_id"].nunique()

Количество участников теста, которые совершали события, почти в 2 раза меньше, чем общее число участников. Посмотрим на распределение участников по группам среди пользователей, которые совершали события.

In [ ]:
events[["user_id", "group"]].drop_duplicates()["group"].value_counts()

А вот здесь уже очень большая разница в количестве пользователей – в 3 раза. Причем в группах разное соотношение между общим количеством пользователей и тех, кто совершал действия.

In [ ]:
events[["user_id", "group"]].drop_duplicates()["group"].value_counts() / participants["group"].value_counts()

Из группы A действия совершили 71% участников, а вот из группы B только 32%, что странно. Однако это вряд ли влияние новой рекомендательной системы, так как она должна влиять только на пользователей, которые совершают действия. Вполне возможно, что у нас неполные данные о действиях пользователей, либо в сборе данных есть какая-то ошибка.

Так как мы ожидаем изменение метрик за 14 дней с момента регистрации, то отбросим все события после 14 дней с момента регистрации пользователя. Но надо учитывать, что для пользователей, которые пришли после 16 декабря 14 дней еще не прошло, и они могут понижать настоящую конверсию. Посмотрим, сколько таких пользователей.

In [ ]:
events.query("first_date > '2020-12-16'")[["user_id", "group"]].drop_duplicates().groupby("group").count()

Больше 30% пользователей группы B и больше 50% группы А зарегистрировались после 16 декабря 2020 года. Если их отбросить, то от предполагаемых 6000 участников останется ~1700, то есть меньше 30%. Тогда, даже если между группами есть статистически значимая разница, статистический тест этого не покажет. Если их оставить, то рассматривать промежуток в 14 дней будет некорректно – конверсии могут не соответствовать реальным.

Оптимальным решением будет посмотреть, за какой промежуток времени в среднем пользователи конвертируются в покупателей. Если этот период меньше, чем 9 дней, тогда можно уменьшить рассматриваемый промежуток времени с 14 до 9 дней.

## Итоги предобработки

Последний зарегистрированный пользователь был 23 декабря, а последнее событие 30 декабря 2020 года, при том что в среднем в день регистрируется 3.5 тысячи новых пользователей и совершается 27 тысяч событий. Это может говорить о том, что данные неполные.

Неполные данные могут поставить под сомнения корректность теста, так как с полными данными статистическая значимость теста может поменяться.

Пропуски есть только в таблице `events` в столбце `details`, но они обозначают, что дополнительной информации о событии нет, так что они оставлены без изменений. 

Дубликатов в данных нет, однако 34% событий произошли одновременно – вероятно, ошибка в логировании.

Столбцы с группой, устройством и регионом пользователя были приведены к категориальному типу данных, что уменьшило потребление памяти.

Было отброшено 12% участников, так как они одновременно присутствовали в другом тесте. Еще 5% участников были не из стран Евросоюза. Среди оставшихся участников только половина совершала действия, причем 75% из них - пользователи группы А. Таким образом между группами присутствует большой дисбаланс в количестве участников, совершавших действия.

Кроме того, больше 30% пользователей группы B и больше 50% группы А зарегистрировались после 16 декабря 2020 года, и с момента их регистрации прошло меньше 14 дней. Если их отбросить, то от предполагаемых 6000 участников останется ~1700, то есть меньше 30%. Тогда, даже если между группами есть статистически значимая разница, статистический тест этого не покажет. Если их оставить, то рассматривать промежуток в 14 дней будет некорректно – конверсии могут не соответствовать реальным.

Оптимальным решением будет посмотреть, за какой промежуток времени в среднем пользователи конвертируются в покупателей. Если этот период меньше, чем 9 дней, тогда можно уменьшить рассматриваемый промежуток времени с 14 до 9 дней.

# Исследовательский анализ данных

In [ ]:
def get_date_distribution(data, date_column, color_column, title, show_marginal=False):
    fig = px.histogram(
        data,
        x=date_column,
        color=color_column,
        histnorm="probability",
        barmode="overlay",
        marginal="box" if show_marginal else None,
    )

    fig.update_layout(
        title=title,
        xaxis_title=None,
        yaxis_title=None,
        xaxis_tickformat="%b %d<br>%a",
        yaxis_tickformat="p",
        xaxis_showgrid=True,
    )

    legend_title = "Устройство:" if color_column == "device" else "Группа теста:"

    fig.update_layout(
        legend=dict(
            title=legend_title,
            orientation="h",
            xanchor="right",
            x=1,
            yanchor="bottom",
            y=1,
        )
    )

    return fig

In [ ]:
color_column_widget = widgets.Dropdown(
    options=[("Группа теста", "group"), ("Устройство", "device")], description="Цвет"
)

interact(
    get_date_distribution,
    data=fixed(participants),
    date_column=fixed("first_date"),
    color_column=color_column_widget,
    title=fixed("Распределение даты регистрации пользователей"),
    show_marginal=fixed(False),
);

Регистрация имеет какую-то цикличную активность, и не зависит от устройства или группы пользователя. Больше всего пользователей регистрируется ближе к понедельнику и меньше всего в середине недели.

In [ ]:
def get_device_distribution():
    fig = px.histogram(events, y="device", histnorm="probability", text_auto=".2p")

    fig.update_layout(
        title="Распределение устройств пользователей",
        xaxis_title=None,
        yaxis_title=None,
        yaxis_categoryorder="total ascending",
        yaxis_linecolor="black",
        xaxis_showgrid=False,
        xaxis_showticklabels=False,
    )

    return fig

In [ ]:
get_device_distribution()

Строго говоря, параметр `device` соответствует устройству, с которого происходила регистрация пользователя. Вполне возможно, что пользователь потом все свои сессий совершал с других устройств. Но будем предполагать, что в большинстве случаев пользователь совершает действия с того же устройства, с которого проходил регистрацию.

Практически половина пользователей заходят к нам с мобильных устройств Android. С компьютера заходят 25% пользователей, а с айфона 21%. С макбуков заходит меньше 10% пользователей.

In [ ]:
interact(
    get_date_distribution,
    data=fixed(events),
    date_column=fixed("event_dt"),
    color_column=color_column_widget,
    title=fixed("Активность пользователей"),
    show_marginal=fixed(True),
);

Активность пользователей тоже неравномерна и сосредоточена вокруг 14-21 декабря. Возможно, в это время люди выбирали подарки к Новому году на нашем сайте. Однако эта активность выглядит весьма странно – резкий скачок активности почти в 3 раза 14 декабря и резкое снижение после 21 декабря. Также странно выглядит резкий обрыв активности после 29 декабря. Ближе к 30 декабря активность идет на убыль – вероятно, люди готовятся к Новому году.

Между устройствами сильной разницы нет, а вот распределение группы B отличается от распределения группы А. Пользователи группы B были активны и до 15 декабря, а у пользователей группы A там минимальная активность. Однако после 15 декабря у пользователей группы А выше активность. Возможно, у нас неполные данные для группы А, либо тут какая-то аномалия.

Посмотрим, сколько часов в среднем проходит до первой покупки.

In [ ]:
def get_first_purchase_hour_distribution():
    first_purchase = events.query("event_name == 'purchase'").groupby("user_id")["event_dt"].min()
    first_date = events.query("event_name == 'purchase'").groupby("user_id")["first_date"].first()
    first_purchase_hour = (first_purchase - first_date).dt.total_seconds() / 3600

    fig = px.histogram(first_purchase_hour, histnorm="probability")

    fig.update_layout(
        title="Распределение количества часов до первой покупки",
        xaxis_title=None,
        yaxis_title=None,
        xaxis_showgrid=True,
        yaxis_tickformat="p",
        showlegend=False,
    )

    return fig

In [ ]:
get_first_purchase_hour_distribution()

В среднем, если пользователи совершают покупку, то в первые 20 часов. Максимальный срок совершения покупки – 7 дней, что все равно меньше, чем 9 дней. Таким образом, можно сократить рассматриваемый промежуток с 14 дней до 9 дней. Если же оставлять 14 дней, то смысла что-либо проверять мало, потому что в группе B будет всего 530 пользователей, вместо предполагаемых 3000 тысяч.

In [ ]:
events["lifetime"] = (events["event_dt"] - events["first_date"]).dt.days
events = events.loc[events["lifetime"] < 9]
events[["user_id", "group"]].drop_duplicates().groupby("group")["user_id"].count()

Количество участников осталось тем же (вдруг кто-то совершил первое событие только через 9 дней после регистрации).

## Воронка событий

Создадим профили пользователей.

In [ ]:
event_names = ["login", "product_page", "product_cart", "purchase"]
profiles = events.pivot_table(index="user_id", columns="event_name", values="event_dt", aggfunc="count")
profiles["revenue"] = events.groupby("user_id")["details"].sum().values
profiles = profiles.merge(participants, on="user_id")
profiles = profiles[["user_id", "group", "device", "first_date", *event_names, "revenue"]]
profiles.head()

In [ ]:
funnel_data = profiles.groupby("group").apply(lambda group: group.astype(bool).sum())
funnel_data = funnel_data[event_names]
funnel_data

In [ ]:
def get_group_funnels(percent_mode="previous", interactive=False):

    fig = make_subplots(1, 2, subplot_titles=["A", "B"], shared_yaxes=True)

    event_names = funnel_data.columns.str.replace("_", " ").str.capitalize()

    funnel_a = px.funnel(x=funnel_data.loc["A"], y=event_names)
    funnel_b = px.funnel(x=funnel_data.loc["B"], y=event_names, color_discrete_sequence=["#EF553B"])

    fig.add_trace(funnel_a.data[0], row=1, col=1)
    fig.add_trace(funnel_b.data[0], row=1, col=2)

    fig.update_layout(title="Воронки тестовых групп", title_x=0.5)

    fig.update_traces(textfont_color="white")

    def update_percent_mode(percent_mode=percent_mode):
        if percent_mode == "previous":
            fig.update_traces(texttemplate="%{percentPrevious}")
        else:
            fig.update_traces(texttemplate="%{percentInitial}")

    update_percent_mode()

    if interactive:

        fig = go.FigureWidget(fig)

        percent_widget = widgets.Dropdown(
            options=[("от предыдущего", "previous"), ("от общего", "total")], description="Процент"
        )

        interact(update_percent_mode, percent_mode=percent_widget)

    return fig

In [ ]:
get_group_funnels(interactive=True)

У группы B ниже доля тех, кто переходил на страницу продукта, но немного выше доля тех, кто потом перешел в корзину. Причем процент покупателей от тех, кто зашел в корзину, больше 100% – значит, покупку можно совершить и не заходя в корзину. Увеличение конверсии на 10% не наблюдается ни на одном из этапов воронки. Общая доля покупателей в группе B на 3% ниже, чем в группе А. 

Новый год мог повлиять на склонность пользователей к покупкам, но эта склонность присутствовала бы в обеих группах и новая система рекомендаций должна была бы только помочь людям быстрее купить подарки/вещи. Но мы наблюдаем только спад конверсии пользователей в покупателей.

## Количество событий на одного пользователя

In [ ]:
def get_events_per_user_distribution(event):
    fig = px.histogram(profiles, x=event, color="group", histnorm="probability", barmode="overlay", marginal="box")

    fig.update_layout(
        title=f'Распределение события <span style="color:#636EFA">{event}</span> по группам',
        xaxis_title=None,
        yaxis_title=None,
        xaxis_showgrid=True,
        yaxis_tickformat="p",
    )

    fig.update_layout(
        legend=dict(
            title="Группа теста:",
            orientation="h",
            xanchor="right",
            x=1,
            yanchor="bottom",
            y=1,
        )
    )

    return fig

In [ ]:
event_widget = widgets.Dropdown(options=["login", "product_page", "product_cart", "purchase"], description="Событие")
interact(get_events_per_user_distribution, event=event_widget);

Распределения количества событий на пользователя весьма похожи, но у группы B оно чуть смещено влево, то есть пользователи группы B делают чуть меньше событий, чем пользователи группы A.

## Выручка на покупателя

In [ ]:
def get_revenue_distribution(color_column):

    fig = px.histogram(
        profiles.query("revenue > 0"),
        x="revenue",
        color=color_column,
        histnorm="probability",
        barmode="overlay",
        marginal="box",
    )

    fig.update_layout(
        title="Распределение выручки на покупателя",
        xaxis_title=None,
        yaxis_title=None,
        xaxis_rangeslider_visible=True,
        xaxis_showgrid=True,
        xaxis_tickprefix="$",
        yaxis_tickformat="p",
    )

    legend_title = "Группа:" if color_column == "group" else "Устройство:"

    fig.update_layout(
        legend=dict(
            title=legend_title,
            orientation="h",
            xanchor="right",
            x=1,
            yanchor="bottom",
            y=1,
        )
    )

    return fig

In [ ]:
interact(get_revenue_distribution, color_column=color_column_widget);

Пользователи разных групп и разных устройств имеют примерно одинаковые распределения выручек. Однако распределение группы B немного левее, чем распределение группы А. Медиана для группы B равна 15\\$, а группы A - 20\\$.

## Итоги по анализу данных

**Общее:**

Регистрация имеет цикличную активность и не зависит от устройства или группы пользователя. Больше всего пользователей регистрируется ближе к понедельнику и меньше всего в середине недели.

Практически половина пользователей заходят к нам с мобильных устройств Android. С компьютера заходят 25% пользователей, а с айфона 21%. С макбуков заходит меньше 10% пользователей.

Активность пользователей тоже неравномерна и сосредоточена вокруг 14-21 декабря. Возможно, в это время люди выбирали подарки к Новому году на нашем сайте. Однако эта активность выглядит весьма странно – резкий скачок активности почти в 3 раза 14 декабря и резкое снижение после 21 декабря. Также странно выглядит резкий обрыв активности после 29 декабря. Ближе к 30 декабря активность идет на убыль – вероятно, люди готовятся к Новому году.

Между устройствами сильной разницы нет, а вот распределение группы B отличается от распределения группы А. Пользователи группы B были активны и до 15 декабря, а у пользователей группы A там минимальная активность. Однако после 15 декабря у пользователей группы А выше активность. Возможно, у нас неполные данные для группы А, либо тут какая-то аномалия.

**Воронка событий:**

У группы B ниже доля тех, кто переходил на страницу продукта, но немного выше доля тех, кто потом перешел в корзину. Причем процент покупателей от тех, кто зашел в корзину, больше 100% – значит, покупку можно совершить и не заходя в корзину. Увеличение конверсии на 10% не наблюдается ни на одном из этапов воронки. Общая доля покупателей в группе B на 3% ниже, чем в группе А. 

Новый год мог повлиять на склонность пользователей к покупкам, но эта склонность присутствовала бы в обеих группах и новая система рекомендаций должна была бы только помочь людям быстрее купить подарки/вещи. Но мы наблюдаем только спад конверсии пользователей в покупателей.

**Количество событий и выручка на пользователя:**

Распределения количества событий на пользователя весьма похожи, но у группы B оно чуть смещено влево, то есть пользователи группы B делают чуть меньше событий, чем пользователи группы A.

Пользователи разных групп и разных устройств имеют примерно одинаковые распределения выручек. Однако распределение группы B немного левее, чем распределение группы А. Медиана для группы B равна 15\\$, а группы A - 20\\$.

# Проверка гипотез

## Конверсия на различных этапах воронки

Проверим, есть ли между группами статистически значимая разница в конверсии пользователей на различных этапах воронки.

**Нулевая гипотеза:** конверсия пользователей на этапе воронки `S` между группами A и B одинакова.

**Альтернативная гипотеза:** конверсия пользователей на этапе воронки `S` между группами A и B различна.

Где `S` - это последовательно `product_page`, `product_cart`, `purchase`. Таким образом, будет проверено 3 гипотезы.

Конверсии будем проверять относительные - то есть от не от общего количества пользователей, а от количества пользователей прошлого этапа воронки. Так мы сможем более точно определить, где именно поменялась конверсия пользователей. Но для покупок конверсию будем проверять общую, потому что, как видно выше, покупку можно совершить, не переходя в корзину.

Для проверки гипотез будем использовать t-тест с уровнем статистической значимости 0.01.

In [ ]:
profiles_bool = profiles.copy(deep=True)
profiles_bool[event_names] = profiles_bool[event_names].astype(bool)

In [ ]:
for event_id in range(5, 7):
    event_a = profiles_bool[profiles_bool.iloc[:, event_id - 1] == True].query("group == 'A'").iloc[:, event_id]
    event_b = profiles_bool[profiles_bool.iloc[:, event_id - 1] == True].query("group == 'B'").iloc[:, event_id]

    pvalue = stats.ttest_ind(event_a, event_b).pvalue

    print(event_names[event_id - 4], f"{pvalue:.2g}")

purchase_a = profiles_bool.query("group == 'A'")["purchase"]
purchase_b = profiles_bool.query("group == 'B'")["purchase"]

purchase_pvalue = stats.ttest_ind(purchase_a, purchase_b).pvalue

print(f"purchase {purchase_pvalue:.2g}")

Для перехода на страницу продукта p-значение достаточно мало, чтобы отвергнуть нулевую гипотезу и сказать, что конверсия перехода на страницу продукта между группами действительно отличается. Только не в пользу тестовой группы, так как у контрольной группы конверсия 65%, а у тестовой – 56%.

Для других этапов воронки p-значения слишком высокие, чтобы отвергать нулевые гипотезы. Это может быть прямым следствием того, что у нас слишком мало участников в группе B, и меньше предполагаемого количества в группе A.

В итоге получается, что новая система рекомендаций статистически значимо хуже в конверсии пользователей.

## Выручка на пользователя

Проверим, есть ли между группами статистически значимая разница в выручке на покупателя.

**Нулевая гипотеза:** выручка на покупателя для групп A и B одинакова.

**Альтернативная гипотеза:** выручка на покупателя для групп A и B различна.

In [ ]:
revenue_a = profiles.query("revenue > 0 and group == 'A'")["revenue"]
revenue_b = profiles.query("revenue > 0 and group == 'B'")["revenue"]

In [ ]:
stats.ttest_ind(revenue_a, revenue_b)

P-значение снова слишком высоко, чтобы отвергнуть нулевую гипотезу и сказать, что средняя выручка с покупателя для групп A и B различна.

# Итог

**Описание данных:**

Вероятнее всего, данные неполные и в сборе данных присутствует ошибка. На это есть следующие причины:

* Последний зарегистрированный пользователь был 23 декабря, а последнее событие 30 декабря 2020 года, при том что в среднем в день регистрируется 3.5 тысячи новых пользователей и совершается 27 тысяч событий. Это может говорить о том, что данные неполные.


* Больше трети событий произошли одновременно – вероятно, ошибка в логировании.


* Среди пользователей, которые совершали действия, пользователей группы A в 3 раза больше, чем пользователей группы B. Причем это вряд ли влияние новой системы рекомендаций, так как она должна влиять только на пользователей, которые совершают действия.


* Пользователи группы B были активны до 15 декабря, а у пользователей группы A там минимальная активность.

Неполные данные могут поставить под сомнения корректность теста, так как с полными данными статистическая значимость теста может поменяться.


**Предобработка данных:**

Пропуски есть только в таблице `events` в столбце `details`, но они обозначают, что дополнительной информации о событии нет, так что они оставлены без изменений.

Кроме одновременных событий, дубликатов в данных нет.

Было отброшено 12% участников, так как они одновременно присутствовали в другом тесте. Еще 5% участников были не из стран Евросоюза. Среди оставшихся участников только половина совершала действия, причем 75% из них - пользователи группы А. Таким образом между группами присутствует большой дисбаланс в количестве участников, совершавших действия.

Кроме того, больше 30% пользователей группы B и больше 50% группы А зарегистрировались после 16 декабря 2020 года, и с момента их регистрации прошло меньше 14 дней. Если их отбросить, то от предполагаемых 6000 участников останется ~1700, то есть меньше 30%. Тогда, даже если между группами есть статистически значимая разница, статистический тест этого не покажет. Если их оставить, то рассматривать промежуток в 14 дней будет некорректно – конверсии могут не соответствовать реальным.

Оптимальным решением будет посмотреть, за какой промежуток времени в среднем пользователи конвертируются в покупателей. Если этот период меньше, чем 9 дней, тогда можно уменьшить рассматриваемый промежуток времени с 14 до 9 дней.


**Исследовательский анализ данных:**

Регистрация имеет цикличную активность и не зависит от устройства или группы пользователя. Больше всего пользователей регистрируется ближе к понедельнику и меньше всего в середине недели.

Практически половина пользователей заходят к нам с мобильных устройств Android. С компьютера заходят 25% пользователей, а с айфона 21%. С макбуков заходит меньше 10% пользователей.

Активность пользователей тоже неравномерна и сосредоточена вокруг 14-21 декабря. Возможно, в это время люди выбирали подарки к Новому году на нашем сайте. Однако эта активность выглядит весьма странно – резкий скачок активности почти в 3 раза 14 декабря и резкое снижение после 21 декабря. Также странно выглядит резкий обрыв активности после 29 декабря. Ближе к 30 декабря активность идет на убыль – вероятно, люди готовятся к Новому году.

Между устройствами сильной разницы нет, а вот распределение группы B отличается от распределения группы А. Пользователи группы B были активны и до 15 декабря, а у пользователей группы A там минимальная активность. Однако после 15 декабря у пользователей группы А выше активность. Возможно, у нас неполные данные для группы А, либо тут какая-то аномалия.

В среднем, если пользователи совершают покупку, то в первые 20 часов. Максимальный срок совершения покупки – 7 дней, что все равно меньше, чем 9 дней. Таким образом, можно сократить рассматриваемый промежуток с 14 дней до 9 дней. Если же оставлять 14 дней, то смысла что-либо проверять нет, потому что в группе B будет всего 450 пользователей, вместо предполагаемых 3000 тысяч.

У группы B значительно ниже доля тех, кто переходил на страницу продукта, но немного выше доля тех, кто потом перешел в корзину. Причем процент покупателей от тех, кто зашел в корзину, больше 100% – значит, покупку можно совершить и не заходя в корзину. Увеличение конверсии на 10% не наблюдается ни на одном из этапов воронки. Общая доля покупателей в группе B на 3% ниже, чем в группе А. 

Новый год мог повлиять на склонность пользователей к покупкам, но эта склонность присутствовала бы в обеих группах и новая система рекомендаций должна была бы только помочь людям быстрее купить подарки/вещи. Но мы наблюдаем только спад конверсии пользователей в покупателей.

Распределения количества событий на пользователя весьма похожи, но у группы B оно чуть смещено влево, то есть пользователи группы B делают чуть меньше событий, чем пользователи группы A.

Пользователи разных групп и разных устройств имеют примерно одинаковые распределения выручек. Однако распределение группы B немного левее, чем распределение группы А. Медиана для группы B равна 15\\$, а группы A - 20\\$.


**Проверка гипотез:**

Было проверено 3 гипотезы, есть ли между группами статистически значимая разница в конверсии пользователей на разных этапах воронки. Для проверки использовался t-тест с уровнем статистической значимости 0.01. 

Для перехода на страницу продукта p-значение достаточно мало, чтобы отвергнуть нулевую гипотезу и сказать, что конверсия перехода на страницу продукта между группами действительно отличается. Только не в пользу тестовой группы, так как у контрольной группы конверсия 65%, а у тестовой – 56%.

Для других этапов воронки p-значения слишком высокие, чтобы отвергать нулевые гипотезы. Это может быть прямым следствием того, что у нас слишком мало участников в группе B, и меньше предполагаемого количества в группе A.

В итоге получается, что новая система рекомендаций статистически значимо хуже в конверсии пользователей.

Также была проверена гипотеза, что средняя выручка с покупателя для групп A и B отличается. P-значение снова было слишком высоким, чтобы отвергнуть нулевую гипотезу.


**Итоги по тестированию:**

Для анализа теста были предоставлены данные низкого качества, а в организации теста присутствуют проблемы, описанные выше. Однако смысла проводить тест повторно, скорее всего, нет, так как тестовая группа не показала увеличения конверсии на 10% ни на одном из этапов воронки, а конверсия пользователей в просмотр товаров даже статистически хуже для тестовой группы. Также не изменилось количество событий на пользователя. Медианная выручка с покупателя у группы B на 5$ меньше, чем у группы А, но статистически значимой разницы обнаружено не было. Вероятнее всего, новая система рекомендаций действительно только понизила конверсию и среднюю выручку с покупателя, так что нет смысла продолжать ее тестировать.